<a href="https://colab.research.google.com/github/alisonnnnn88/programming_language/blob/main/HW5_%E9%A3%AF%E5%BA%97%E6%9F%A5%E8%A9%A2%E5%9C%B0%E5%9C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install folium geopy gradio pandas requests -q

import requests
import folium
import pandas as pd
import gradio as gr
import random
from geopy.geocoders import Nominatim

# ==========================
# 初始化地理編碼器
# ==========================
geolocator = Nominatim(user_agent="hotel_search_app")

def get_coordinates(location_name):
    try:
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except:
        return None

# ==========================
# 抓附近飯店資料
# ==========================
def fetch_hotels(lat, lon, radius=2000):
    query = f"""
    [out:json];
    node
      [tourism~"hotel|motel|guest_house"]
      (around:{radius},{lat},{lon});
    out;
    """
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data={'data': query})
    data = response.json()

    hotels = []
    for el in data.get("elements", []):
        tags = el.get("tags", {})
        hotels.append({
            "飯店名稱": tags.get("name", "未知飯店"),
            "評價": round(random.uniform(3.0,5.0),1),
            "價位": random.randint(2000,8000),
            "地址": tags.get("addr:street", "無地址"),
            "緯度": el["lat"],
            "經度": el["lon"]
        })
    # 若無資料，補模擬飯店
    if not hotels:
        for i in range(5):
            lat_jitter = lat + random.uniform(-0.02,0.02)
            lon_jitter = lon + random.uniform(-0.02,0.02)
            hotels.append({
                "飯店名稱": f"模擬飯店{i+1}",
                "評價": round(random.uniform(3.0,5.0),1),
                "價位": random.randint(2000,8000),
                "地址": f"{lat_jitter},{lon_jitter}",
                "緯度": lat_jitter,
                "經度": lon_jitter
            })
    return pd.DataFrame(hotels)


# ==========================
# 篩選 & AI 推薦
# ==========================
def search_and_recommend(location, min_rating, max_price, checkin, checkout):
    coords = get_coordinates(location)
    if not coords:
        return None, "<p>無法取得地點座標</p>", "請輸入正確地名", None
    lat, lon = coords

    df_hotels = fetch_hotels(lat, lon)
    if df_hotels.empty:
        return None, "<p>此地區找不到飯店</p>", "（無建議）", None

    # 篩選條件
    filtered = df_hotels[(df_hotels['評價'] >= min_rating) & (df_hotels['價位'] <= max_price)]
    if filtered.empty:
        filtered = df_hotels.sample(min(3,len(df_hotels)))

    # AI 推薦三家
    recommended = filtered.sample(min(3,len(filtered)))
    ai_summary = "AI 建議：可考慮 " + "、".join([f"{row['飯店名稱']} ⭐{row['評價']} 💰{row['價位']}" for _, row in recommended.iterrows()])
    ai_summary += f"（入住：{checkin}，退房：{checkout}）"

    # 建立地圖
    m = folium.Map(location=[lat, lon], zoom_start=13 if len(filtered) < 30 else 11)
    m.fit_bounds([[filtered["緯度"].min(), filtered["經度"].min()],
              [filtered["緯度"].max(), filtered["經度"].max()]])

    # 顏色依評價顯示
    def get_color(rating):
        if rating >= 4.5:
            return "green"
        elif rating >= 4.0:
            return "blue"
        elif rating >= 3.5:
            return "orange"
        else:
            return "red"

    for _, row in filtered.iterrows():
      icon_color = "purple" if row["飯店名稱"] in recommended["飯店名稱"].values else get_color(row["評價"])
      folium.Marker(
          [row["緯度"], row["經度"]],
          popup=f"<b>{row['飯店名稱']}</b><br>⭐ {row['評價']} | 💰 {row['價位']}<br>{row['地址']}",
          icon=folium.Icon(color=icon_color, icon="star" if icon_color=="purple" else "home")
      ).add_to(m)

    return m._repr_html_(), filtered[['飯店名稱','評價','價位','地址']], ai_summary, recommended

# ==========================
# Gradio 介面
# ==========================
TAIWAN_CITIES = [
    "台北市", "新北市", "基隆市", "桃園市", "新竹市", "新竹縣",
    "苗栗縣", "台中市", "彰化縣", "南投縣", "雲林縣", "嘉義市", "嘉義縣",
    "台南市", "高雄市", "屏東縣", "宜蘭縣", "花蓮縣", "台東縣",
    "澎湖縣", "金門縣", "連江縣"
]

with gr.Blocks(title="台灣飯店搜尋地圖＋AI推薦") as demo:
    gr.Markdown("## 🏨 台灣飯店搜尋地圖 + AI 推薦")

    with gr.Tab("🔍 地圖搜尋"):
        with gr.Row():
            city_input = gr.Dropdown(label="目的地", choices=TAIWAN_CITIES, value="台北市")
            rating_slider = gr.Slider(0,5,value=3.5,step=0.1,label="最低評價")
            price_slider = gr.Slider(1000,20000,value=8000,step=500,label="價位上限 (NTD)")
        with gr.Row():
            checkin_input = gr.Textbox(label="入住日期 (YYYY-MM-DD)")
            checkout_input = gr.Textbox(label="退房日期 (YYYY-MM-DD)")
        search_btn = gr.Button("🔎 搜尋飯店")

        map_output = gr.HTML(label="地圖")
        table_output = gr.DataFrame(label="搜尋結果")

        search_btn.click(
            fn=search_and_recommend,
            inputs=[city_input,rating_slider,price_slider,checkin_input,checkout_input],
            outputs=[map_output, table_output, gr.Textbox()]
        )

    with gr.Tab("🤖 AI 推薦"):
        condition_input = gr.Textbox(label="輸入想要的條件（例如：有泳池、含早餐）")
        ai_output = gr.Textbox(label="AI 推薦結果")

        def dynamic_ai_recommend(condition, recommended_df):
            if recommended_df is None or recommended_df.empty:
                return "⚠️ 目前沒有搜尋結果可推薦"
            hotel_list = "、".join(recommended_df['飯店名稱'].tolist())
            return f"根據你的需求「{condition}」以及搜尋結果，推薦考慮：{hotel_list}。"

        gr.Button("✨ 生成建議").click(
            fn=dynamic_ai_recommend,
            inputs=[condition_input, table_output],  # 將搜尋結果傳入
            outputs=ai_output
        )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dbef1517276c9a2529.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
